# FFT response or ODS generation
A Notebook for generating FFT or ODS of operational response after data acquisition.

In [ ]:
from lvm_read import *
from functions import *

In [ ]:
folder_name = "./data/"
imp_loc = os.listdir(folder_name)
imp_loc = [value for value in imp_loc if ".npy" in value]
imp_loc = sorted_alphanumeric(imp_loc)

filename_T = folder_name + imp_loc[0] 
_list_data = np.load(filename_T,allow_pickle=True)

#### FFT generation

In [ ]:
_out = _list_data[0]["data"].shape[1]-1
_in = len(imp_loc)

_freq = 2000

H_fft = np.zeros((2000,_out,_in),dtype = complex)
H_time = np.zeros((25600,_out,_in))


for _in_j,imp in enumerate(imp_loc):
    print(imp)
    filename_T = folder_name + imp 

    _list_data = np.load(filename_T,allow_pickle=True)
    no_impact = len(_list_data)
    no_sensors = _list_data[0]["data"].shape[1]-1
        
    
    for _out_i in range(no_sensors):
        
        sum_SFX = 0
        sum_SFF = 0
        sum_SXX = 0
        sum_SXF = 0

        for p,data in enumerate(_list_data):

            f, F = FFT_R_F(data["sample_rate"], data["data"][:,0])
            f, X = FFT_R_X(data["sample_rate"], data["data"][:,_out_i+1])
            fs = data["sample_rate"]
            
            H_time[:,_out_i,_in_j] = data["data"][:,_out_i+1]

            H1 = X  

        freq = f[np.where(f<2000)]
        H_fft[:,_out_i,_in_j] = H1[np.where(f<2000)]
                


In [ ]:
try:
    os.mkdir(folder_name+"FRFs/")
except:
    pass
np.save(folder_name+"FRFs/"+"freq",freq)
np.save(folder_name+"FRFs/"+"H_fft",H_fft)


In [ ]:
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets

out_list = [i for i in range(_out)]
in_list = [i for i in range(_in)]

selection_out = widgets.SelectMultiple(
        options=out_list,
        value=out_list,
        rows = len(out_list),
        description = "Out"
    )

selection_in = widgets.SelectMultiple(
        options=in_list,
        value=in_list,
        rows = len(in_list),
        description = "In"
    )


button = widgets.Button(description="Plot")
output = widgets.Output()


def on_button_clicked(B):
    with output:
        clear_output()
        lst_in = list(selection_in.value)
        lst_out = list(selection_out.value)
        
        fig = plt.figure(figsize =(8,6))
        plt.subplot(211)
        for select_in in lst_in:
            for select_out in lst_out:
                plt.semilogy(freq,np.abs(H_fft[:,select_out,select_in]),linewidth = 0.6)
        plt.xlim(freq[0],freq[-1])
        

        plt.subplot(413)
        for select_in in lst_in:
            for select_out in lst_out:
                plt.plot(freq,np.angle(H_fft[:,select_out,select_in]),linewidth = 0.6)
        
        plt.xlim(freq[0],freq[-1])
        plt.show()


button.on_click(on_button_clicked)
display(selection_out,selection_in, button, output)


#### ODS generation

In [ ]:
from scipy.signal import csd
from scipy import signal

In [ ]:
# define ODS function! 
# freq, out, in - not force but displacements

_,out,p_in = H_time.shape
H_ods = np.zeros((2000,out,p_in),dtype = complex)

win = "hamming" #"boxcar"

# select a reference channels
ref_select = 0
for _o in range(out):
    for _i in range(p_in):

        # xps - between response and reference response
        f, xps = csd(H_time[:,_o,_i], H_time[:,ref_select,_i],fs = fs,nperseg = 25600,noverlap = 0,window = win,scaling = "spectrum")
        # aps - at the response 
        f, aps = csd(H_time[:,_o,_i], H_time[:,_o,_i],fs = fs,nperseg = 25600,noverlap = 0,window = win,scaling = "spectrum")

        # ODS = magnitude from aps + phase from the xps 
        ods = np.abs(aps**0.5)*np.cos(np.angle(xps)) + 1j*np.abs(aps**0.5)*np.sin(np.angle(xps))
        
        H_ods[:,_o,_i] = ods[:2000]

In [ ]:
H_ods.shape

In [ ]:
try:
    os.mkdir(folder_name+"FRFs/")
except:
    pass
np.save(folder_name+"FRFs/"+"freq",freq)
np.save(folder_name+"FRFs/"+"H_ods",H_ods)


In [ ]:
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets

out_list = [i for i in range(_out)]
in_list = [i for i in range(_in)]

selection_out = widgets.SelectMultiple(
        options=out_list,
        value=out_list,
        rows = len(out_list),
        description = "Out"
    )

selection_in = widgets.SelectMultiple(
        options=in_list,
        value=in_list,
        rows = len(in_list),
        description = "In"
    )


button = widgets.Button(description="Plot")
output = widgets.Output()


def on_button_clicked(B):
    with output:
        clear_output()
        lst_in = list(selection_in.value)
        lst_out = list(selection_out.value)
        
        fig = plt.figure(figsize =(8,6))
        plt.subplot(211)
        for select_in in lst_in:
            for select_out in lst_out:
                plt.semilogy(freq,np.abs(H_ods[:,select_out,select_in]),linewidth = 0.6)
        plt.xlim(freq[0],freq[-1])
        

        plt.subplot(413)
        for select_in in lst_in:
            for select_out in lst_out:
                plt.plot(freq,np.angle(H_ods[:,select_out,select_in]),linewidth = 0.6)
        
        plt.xlim(freq[0],freq[-1])
        plt.show()


button.on_click(on_button_clicked)
display(selection_out,selection_in, button, output)
